In [2]:
!pip install nltk
!pip install numpy
!pip install scipy
!pip install -U scikit-learn
!pip install scikit-surprise
!pip install pandas
!apt-get install libfreetype6-dev libpng-dev -y
!pip install matplotlib
!pip install seaborn
!pip install jupyter
!pip install jupyterlab
!pip install textblob
!pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357272 sha256=0f49229762bc82fa9b1ce1a63966fb412410ce85e4ecd70767c13ba200fc12c3
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libpng-dev is already the newest version (1.6.37-3build5).
libfreetype6-dev is already the newest version (2.11.1+dfsg-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━

In [3]:
import numpy as np
from lightfm.datasets import fetch_movielens
from lightfm import LightFM

In [4]:
#fetch data and format
data = fetch_movielens(min_rating = 4.0)

#print training and testing data
print(repr(data['train']))
print(repr(data['test']))

<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 49906 stored elements in COOrdinate format>
<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 5469 stored elements in COOrdinate format>


In [5]:
#create model
model = LightFM(loss='warp')
#train model
model.fit(data['train'], epochs = 30, num_threads=2)

In [6]:
def sample_recommendation(model, data, user_ids):

    #number of users and movies in training data
    n_users, n_items = data['train'].shape

    #generate recommendations for each user input
    for user_id in user_ids:

        #movies they always like
        known_positives = data['item_labels'][data['train'].tocsr()[user_id].indices]

        #movies the model predicts they will like
        scores = model.predict(user_id, np.arange(n_items))
        #rank in order of most liked to least
        top_items = data['item_labels'][np.argsort(-scores)]

        #print results
        print("User %s" % user_id)
        print("         Known positives:")

        for x in known_positives[:3]:
            print("          %s" % x)

        print("         Recommend:")

        for x in top_items[:3]:
            print("         %s" % x)

In [7]:
sample_recommendation(model, data, [0, 3, 25, 34, 450])

User 0
         Known positives:
          T
          F
          S
         Recommend:
         M
         S
         P
User 3
         Known positives:
          S
          C
          S
         Recommend:
         S
         S
         A
User 25
         Known positives:
          D
          S
          F
         Recommend:
         F
         E
         C
User 34
         Known positives:
          A
          G
          S
         Recommend:
         A
         C
         S
User 450
         Known positives:
          C
          G
          E
         Recommend:
         A
         K
         D


In [8]:
import pickle
with open('movie_rec.pkl', 'wb') as f:
    pickle.dump(model, f)

In [9]:
with open('movie_rec.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

**FRONT END**

In [10]:
!pip install flask-ngrok

In [11]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [12]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, render_template
import pickle
import numpy as np

In [ ]:
from flask import Flask, request, render_template_string
import pickle
import numpy as np
from google.colab import output

app = Flask(__name__)

# Load the model and data
with open('movie_rec.pkl', 'rb') as f:
    model = pickle.load(f)

# HTML template as a string
html_template = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Movie Recommendation System</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            max-width: 800px;
            margin: 0 auto;
            padding: 20px;
        }
        h1 {
            color: #333;
        }
        form {
            margin-bottom: 20px;
        }
        input[type="number"] {
            width: 100px;
        }
        ul {
            list-style-type: none;
            padding: 0;
        }
        li {
            margin-bottom: 5px;
        }
    </style>
</head>
<body>
    <h1>Movie Recommendation System</h1>
    <form action="/recommend" method="post">
        <label for="user_id">Enter User ID:</label>
        <input type="number" id="user_id" name="user_id" required>
        <input type="submit" value="Get Recommendations">
    </form>

    {% if user_id is defined %}
    <h2>Results for User {{ user_id }}</h2>
    <h3>Known Positives:</h3>
    <ul>
        {% for movie in known_positives %}
        <li>{{ movie }}</li>
        {% endfor %}
    </ul>
    <h3>Recommendations:</h3>
    <ul>
        {% for movie in recommendations %}
        <li>{{ movie }}</li>
        {% endfor %}
    </ul>
    {% endif %}
</body>
</html>
'''

@app.route('/')
def home():
    return render_template_string(html_template)

@app.route('/recommend', methods=['POST'])
def recommend():
    user_id = int(request.form['user_id'])
    n_items = data['train'].shape[1]

    # Get known positives
    known_positives = data['item_labels'][data['train'].tocsr()[user_id].indices]

    # Get recommendations
    scores = model.predict(user_id, np.arange(n_items))
    top_items = data['item_labels'][np.argsort(-scores)]

    return render_template_string(html_template,
                                  user_id=user_id,
                                  known_positives=known_positives[:3],
                                  recommendations=top_items[:3])

# Run the Flask app
if __name__ == '__main__':
    output.serve_kernel_port_as_window(5000)  # or use as_window() if you prefer a new window
    app.run(host='0.0.0.0',port=5000)

Try `serve_kernel_port_as_iframe` instead. 


<IPython.core.display.Javascript object>

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [28/Sep/2024 07:04:52] "GET /?authuser=0 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Sep/2024 07:04:54] "GET /favicon.ico?authuser=0 HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [28/Sep/2024 07:06:01] "POST /recommend?authuser=0 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Sep/2024 07:06:01] "GET /favicon.ico?authuser=0 HTTP/1.1" 404 -
